#  Applied electric field simulation

trajectory not included due to size limit

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()

import mdtraj as md
import numpy as np
import os

import MDAnalysis as mda
import nglview as nv

import pandas as pd

import gmxapi as gmx

In [ ]:
import glob

In [ ]:
from scipy import stats

In [ ]:
from manuscript import *

In [ ]:
sns.set(style="ticks",context='paper',font_scale=2)

# Analysis

In [ ]:
from NACHRA7_annotations import *

In [ ]:
traj_note_dic = ef_dic

In [ ]:
traj_note_dic

In [ ]:
class POTENTIAL_XVG(object):
    def __init__(self, location):
        self.filename = location
        self.read()
        
    def read(self):
        self.potential = pd.read_table(self.filename,
                                 header=None,
                                 names=['coord','potential'],
                                 sep='\s+',
                                 error_bad_lines=False,
                                 skiprows=24)
        for column in self.potential:
            self.potential[column] = self.potential[column].astype(float)       
#        self.potential['potential'] = self.potential['potential']

In [ ]:
def create_md_dataframe():    
    return pd.DataFrame(columns=list(['MD_name','replicate','frame','traj_time','system','id','ligand','note']))

In [ ]:
def append_metadata(traj_note, rep, ident, system, location, skip = default_skip):
    rep_data = []
    traj_note_split = traj_note.split('_')
    top_location = '/' + "init.pdb"
    traj_location = '/rep' + rep + '/' + "ef.xtc"
    traj = mda.Universe(location + top_location,
                        location + traj_location)         
    md_name = traj_note_split[0]
    ligand = traj_note_split[1:-1] ##in this case
    timestep = 2
    note = traj_note_split[-1]
    n_frames = traj.trajectory.n_frames
    ts = traj.trajectory.dt
    for i in range(n_frames):
        rep_data.append([md_name, rep, i,  ts * i, system, ident,ligand,note])

    return rep_data
    
    
meta_data = Parallel(n_jobs=num_cores)(delayed(append_metadata)(traj_note = traj_note_dic['traj_note'][0], 
                                                        rep = str(i%4 + 1),
                                                        ident = i,
                                                        system = i//4,
                                                        location = traj_note_dic['save_location'][i//4],
                                                       skip = traj_note_dic['skip'][i//4], 
                                               )
                           for i in range(0, np.sum(traj_note_dic['rep'])))
md_data = create_md_dataframe()
for i in range(0,np.sum(traj_note_dic['rep'])):
    md_data = md_data.append(pd.DataFrame(meta_data[i],columns=list(['MD_name','replicate','frame','traj_time','system','id','ligand','note'])),ignore_index=True)
md_data['frame'] =md_data['frame'].apply(int)
md_data['traj_time'] =md_data['traj_time'].apply(float)
md_data['replicate'] =md_data['replicate'].apply(int)
md_data['system'] = md_data['system'].apply(int)

In [ ]:
def get_potential(rep, location):
    u = mda.Universe(location + '/init.pdb')
    z_dim = u.dimensions[2]
    gmx_run = gmx.commandline_operation('gmx_d',
                      arguments=['potential','-sl','5000','-correct', '-tz', str(-z_dim/2)],
                      input_files={
                            '-f': location + '/rep' + rep + '/' + "ef.xtc",
                            '-s': location + '/ef_double.tpr',
                            '-n': location + '/index.ndx'
                      },
                      output_files={
                            '-o': location + '/rep' + rep + '/' + '/potential.xvg',
                            '-oc': location + '/rep' + rep + '/' + '/charge.xvg',
                            '-of': location + '/rep' + rep + '/' + '/field.xvg'
                      },
                      stdin='0\n'
                     )
    gmx_run.run()
    print(gmx_run.output.erroroutput.result())

    try:
        for f in glob.glob(location + '/rep' + rep + '/'  + '/#*'):
            os.remove(f)
    except:
        pass
    
def append_potential(rep, location):
    potential_data = []
    get_potential(rep, location)
    potential_data.append(POTENTIAL_XVG(location + '/rep' + rep + '/' + '/potential.xvg'))
    return potential_data
    
potention_data = Parallel(n_jobs=num_cores)(delayed(append_potential)(rep = str(i%4 + 1),
                                                        location = traj_note_dic['save_location'][i//4])
                           for i in range(0, np.sum(traj_note_dic['rep'])))
#xvg_data['potential'] = [x for x in np.hstack(potention_data) if x is not None]

In [ ]:
def get_potential(rep, location):
    u = mda.Universe(location + '/init.pdb')
    z_dim = u.dimensions[2]
    gmx_run = gmx.commandline_operation('gmx_d',
                      arguments=['potential','-sl','5000','-correct', '-tz', str(-z_dim/20), '-e', str(10000)],
                      input_files={
                            '-f': location + '/rep' + rep + '/' + "ef.xtc",
                            '-s': location + '/ef_double.tpr',
                            '-n': location + '/index.ndx'
                      },
                      output_files={
                            '-o': location + '/rep' + rep + '/' + '/potential.xvg',
                            '-oc': location + '/rep' + rep + '/' + '/charge.xvg',
                            '-of': location + '/rep' + rep + '/' + '/field.xvg'
                      },
                      stdin='0\n'
                     )
    gmx_run.run()
    print(gmx_run.output.erroroutput.result())

    try:
        for f in glob.glob(location + '/rep' + rep + '/'  + '/#*'):
            os.remove(f)
    except:
        pass
    
def append_potential(rep, location):
    potential_data = []
    get_potential(rep, location)
    potential_data.append(POTENTIAL_XVG(location + '/rep' + rep + '/' + '/potential.xvg'))
    return potential_data
    
potention_data = Parallel(n_jobs=num_cores)(delayed(append_potential)(rep = str(i%4 + 1),
                                                        location = traj_note_dic['save_location'][i//4])
                           for i in range(0, np.sum(traj_note_dic['rep'])))
#xvg_data['potential'] = [x for x in np.hstack(potention_data) if x is not None]

In [ ]:
def append_all_ion_info(ion, traj_note, rep, system, location):
    def cartesian_to_cylinder(posx, posy, princ_x, princ_y):
        return np.sqrt((posx - princ_x) ** 2 + (posy - princ_y) ** 2)
    
    ion_data = []
    traj_note_split = traj_note.split('_')
    top_location = location + '/' + "init.pdb"
    traj_location = location + '/rep' + rep + '/' + "ef.xtc"
    traj = mda.Universe(top_location,
                        traj_location)

    ions = traj.select_atoms('resname ' + ion)


    princ_center_ch = np.mean(traj.select_atoms('resid 247 and name CA').positions[:5],axis=0)

    for ts in traj.trajectory[:md_data[(md_data.system==system)&(md_data.replicate==eval(rep))]['frame'].max()+1]:
        position_x = ions.positions.T[0]
        position_y = ions.positions.T[1]
        position_z = ions.positions.T[2] - princ_center_ch[2]
#        position_r = cartesian_to_cylinder(position_x, position_y, princ_center_ch[0], princ_center_ch[1])

        ion_data.append(np.asarray([position_x,
                                    position_y,
                                    position_z,
                                    ]).T)
    return ion_data


for ion in ['SOD', 'CAL', 'CLA']:    
    ion_data = Parallel(n_jobs=num_cores)(delayed(append_all_ion_info)(
                                                        ion = ion,
                                                        traj_note = traj_note_dic['traj_note'][0], 
                                                        rep = str(i%4 + 1),
                                                        system = int(i/4),
                                                        location = traj_note_dic['save_location'][i//4])
                           for i in range(0, np.sum(traj_note_dic['rep'])))

    ion_data_concat = [x for x in ion_data if x != []]
    md_data[ion + '_data'] = [item for sublist in ion_data_concat for item in sublist]

In [ ]:
system = ['PNU_200', 'PNU_n200', 'PNU_0CA_200', 'PNU_0CA_n200', 'EPJ_200', 'EPJ_n200','PNU_noECD_200', 'PNU_noECD_n200', 'PNU_noICD_200', 'PNU_noICD_n200', 'PNU_E97A_200', 'PNU_E97A_n200']

In [ ]:
for (i, (sys, df)) in enumerate(md_data.groupby(['system'])):
    fig, axes = plt.subplots(1,4, figsize=(80,10), sharex=True, sharey=True, gridspec_kw={'wspace': 0})

    for (ind, df), ax in zip(df.groupby(['replicate']),axes):

        ion_all = df['SOD' + '_data'].apply(lambda x: np.array(x)).to_numpy()
        ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
        for single_ion in ion_arr_all:
            z_coord = single_ion.T[2].ravel()
            mask_thr = z_coord.min() + 10
            time = df['traj_time'].ravel()
            #time = time[np.where(np.logical_and(z_coord>=60, z_coord<=120))]
            time = np.ma.masked_where(z_coord<=mask_thr + 10, time)

            z_coord = np.ma.masked_where(z_coord<=mask_thr + 10, z_coord)
            ax.plot(time,
                    z_coord)

        ax.set_title('SOD rep ' + str(ind))
        ax.set_ylim(-30,30)
    #    ax.set_xlim(0,10000)
    plt.savefig('./Figures/ef_' + system[i] + '.png',bbox_inches = 'tight', pad_inches=0.1, transparent=False)
    #plt.show()


## Estimate Conductance

In [ ]:
threshold_middle = 3
threshold_time = 100

In [ ]:
# Sodium

for (i, ((sys, ind), df)) in enumerate(md_data.groupby(['system','replicate'])):
#    conduct_list = []
    permeation_event = 0
    df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
    time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
    ion_all = df['SOD' + '_data'].apply(lambda x: np.array(x)).to_numpy()
    ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
    for ion_single in ion_arr_all:
        ion_z_single = ion_single.T[2]
        mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
        if mask.any():
            middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
            for middle_p in middle_points:
                try:
                    if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                        permeation_event += 1
                        break
                except:
                    pass
            
    columb_e_conv = 6.242E18
    vmemb = 0.2
    conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
    print('In sys', sys, 'in rep', ind, 'time', time, 'ns, permeation', permeation_event, 'conductance', conductance, 'pS')
#    conduct_list.append(conductance)
#    print('Conductance avg:', np.mean(conductance), '+-', np.var(conductance))
print('')

In [ ]:
for (i, ((sys), sys_df)) in enumerate(md_data.groupby(['system'])):
    conduct_list = []

    for (j, (ind, df)) in enumerate(sys_df.groupby(['replicate'])):
        permeation_event = 0
        df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
        time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
        ion_all = df['SOD' + '_data'].apply(lambda x: np.array(x)).to_numpy()
        ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
        for ion_single in ion_arr_all:
            ion_z_single = ion_single.T[2]
            mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
            if mask.any():
                middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
                for middle_p in middle_points:
                    try:
                        if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                            permeation_event += 1
                            break
                    except:
                        pass

        columb_e_conv = 6.242E18
        vmemb = 0.2
        conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
        #print('In sys', sys, 'in rep', ind, 'time', time, 'ns, permeation', permeation_event, 'conductance', conductance, 'pS')
        conduct_list.append(conductance)
    print(system[sys])
    print('Conductance avg:', np.mean(conduct_list), '+-',  np.std(conduct_list), 'pS')
    print('')

In [ ]:
md_data['direction'] = md_data.system.apply(lambda x: 'up' if x%2 == 0 else 'down')

In [ ]:
# Chloride
for (i, ((sys, ind), df)) in enumerate(md_data.groupby(['system','replicate'])):
    permeation_event = 0
    df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
    time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
    ion_all = df['CLA' + '_data'].apply(lambda x: np.array(x)).to_numpy()
    ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
    for ion_single in ion_arr_all:
        ion_z_single = ion_single.T[2]
        mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
        if mask.any():
            middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
            for middle_p in middle_points:
                try:
                    if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                        permeation_event += 1
                        break
                except:
                    pass
             
    columb_e_conv = 6.242E18
    vmemb = 0.2
    conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
    print('In sys', sys, 'in rep', ind, 'time', time, 'ns, permeation', permeation_event, 'conductance', conductance, 'pS')
print('')

In [ ]:
for (i, ((sys), sys_df)) in enumerate(md_data.groupby(['system'])):
    conduct_list = []

    for (j, (ind, df)) in enumerate(sys_df.groupby(['replicate'])):
        permeation_event = 0
        df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
        time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
        ion_all = df['CLA' + '_data'].apply(lambda x: np.array(x)).to_numpy()
        ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
        for ion_single in ion_arr_all:
            ion_z_single = ion_single.T[2]
            mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
            if mask.any():
                middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
                for middle_p in middle_points:
                    try:
                        if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                            permeation_event += 1
                            break
                    except:
                        pass

        columb_e_conv = 6.242E18
        vmemb = 0.2
        conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
        #print('In sys', sys, 'in rep', ind, 'time', time, 'ns, permeation', permeation_event, 'conductance', conductance, 'pS')
        conduct_list.append(conductance)
    print(system[sys])

    print('Conductance avg:', np.mean(conduct_list), '+-',  np.std(conduct_list), 'pS')
    print('')

In [ ]:
for (i, ((sys), sys_df)) in enumerate(md_data.groupby(['system'])):
    conduct_list = []

    for (j, (ind, df)) in enumerate(sys_df.groupby(['replicate'])):
        permeation_event = 0
        df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
        time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
        ion_all = df['CAL' + '_data'].apply(lambda x: np.array(x)).to_numpy()
        ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
        for ion_single in ion_arr_all:
            ion_z_single = ion_single.T[2]
            mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
            if mask.any():
                middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
                for middle_p in middle_points:
                    try:
                        if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                            permeation_event += 1
                            break
                    except:
                        pass

        columb_e_conv = 6.242E18
        vmemb = 0.2
        conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
        #print('In sys', sys, 'in rep', ind, 'time', time, 'ns, permeation', permeation_event, 'conductance', conductance, 'pS')
        conduct_list.append(conductance)
    print(system[sys])

    print('Conductance avg:', np.mean(conduct_list), '+-',  np.std(conduct_list), 'pS')
    print('')

In [ ]:
md_data.to_pickle('data/ef.pickle')

In [ ]:
md_data = pd.read_pickle('data/ef.pickle')

In [ ]:
def create_md_dataframe():    
    return pd.DataFrame(columns=list(['MD_name','replicate','traj_time','system','id','ligand','note']))

In [ ]:
traj_note_dic

In [ ]:
def append_metadata(traj_note, rep, ident, system, location, skip = default_skip):
    rep_data = []
    traj_note_split = traj_note.split('_')
    top_location = '/' + "init.pdb"
    traj_location = '/rep' + rep + '/' + "ef.xtc"
    traj = mda.Universe(location + top_location,
                        location + traj_location)         
    md_name = traj_note_split[0]
    if location.split('/')[-1] == '': 
        ligand = location.split('/')[-3]
        note = location.split('/')[-2]
    else:
        ligand = location.split('/')[-2]
        note = location.split('/')[-1]        

    timestep = 2
    n_frames = traj.trajectory.n_frames
    ts = traj.trajectory.dt
    rep_data.append([md_name, rep, ts * n_frames, system, ident,ligand,note])

    return rep_data
    
    
meta_data = Parallel(n_jobs=num_cores)(delayed(append_metadata)(traj_note = traj_note_dic['traj_note'][0], 
                                                        rep = str(i%4 + 1),
                                                        ident = i,
                                                        system = i//4,
                                                        location = traj_note_dic['save_location'][i//4],
                                                       skip = traj_note_dic['skip'][i//4], 
                                               )
                           for i in range(0, np.sum(traj_note_dic['rep'])))
ef_data = create_md_dataframe()
for i in range(0,np.sum(traj_note_dic['rep'])):
    ef_data = ef_data.append(pd.DataFrame(meta_data[i],columns=list(ef_data.columns)),ignore_index=True)
ef_data['traj_time'] =ef_data['traj_time'].apply(float)
ef_data['replicate'] =ef_data['replicate'].apply(int)
ef_data['system'] = ef_data['system'].apply(int)

In [ ]:
def get_potential(rep, location):
    u = mda.Universe(location + '/init.pdb')
    z_dim = u.dimensions[2]
    gmx_run = gmx.commandline_operation('gmx_d',
                      arguments=['potential','-sl','5000','-correct', '-tz', str(-z_dim/2)],
                      input_files={
                            '-f': location + '/rep' + rep + '/' + "ef.xtc",
                            '-s': location + '/ef_double.tpr',
                            '-n': location + '/index.ndx'
                      },
                      output_files={
                            '-o': location + '/rep' + rep + '/' + '/potential.xvg',
                            '-oc': location + '/rep' + rep + '/' + '/charge.xvg',
                            '-of': location + '/rep' + rep + '/' + '/field.xvg'
                      },
                      stdin='0\n'
                     )
    gmx_run.run()
    print(gmx_run.output.erroroutput.result())

    try:
        for f in glob.glob(location + '/rep' + rep + '/'  + '/#*'):
            os.remove(f)
    except:
        pass
    
def append_potential(rep, location):
    potential_data = []
    get_potential(rep, location)
    potential_data.append(POTENTIAL_XVG(location + '/rep' + rep + '/' + '/potential.xvg'))
    return potential_data
    
potention_data = Parallel(n_jobs=num_cores)(delayed(append_potential)(rep = str(i%4 + 1),
                                                        location = traj_note_dic['save_location'][i//4])
                           for i in range(0, np.sum(traj_note_dic['rep'])))
ef_data['potential'] = [x for x in np.hstack(potention_data) if x is not None]

In [ ]:
threshold_middle = 3
threshold_time = 100

In [ ]:
conductance_list = []
for (i, ((sys, ind), df)) in enumerate(md_data.groupby(['system','replicate'])):
#    conduct_list = []
    permeation_event = 0
    df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
    time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
    ion_all = df['SOD' + '_data'].apply(lambda x: np.array(x)).to_numpy()
    ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
    for ion_single in ion_arr_all:
        ion_z_single = ion_single.T[2]
        mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
        if mask.any():
            middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
            for middle_p in middle_points:
                try:
                    if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                        permeation_event += 1
                        break
                except:
                    pass
            
    columb_e_conv = 6.242E18
    vmemb = 0.2
    conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
    conductance_list.append(conductance)
ef_data['conductance_sod'] = conductance_list

In [ ]:
conductance_list = []
for (i, ((sys, ind), df)) in enumerate(md_data.groupby(['system','replicate'])):
#    conduct_list = []
    permeation_event = 0
    df = md_data[(md_data.system == sys) & (md_data.replicate == ind)]
    time = md_data[(md_data.system == sys) & (md_data.replicate == ind)].shape[0] * 40 / 1000
    ion_all = df['CLA' + '_data'].apply(lambda x: np.array(x)).to_numpy()
    ion_arr_all = np.concatenate(ion_all).reshape(df.shape[0],ion_all[0].shape[0],3).transpose((1,0,2))
    for ion_single in ion_arr_all:
        ion_z_single = ion_single.T[2]
        mask = (ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle)
        if mask.any():
            middle_points = np.argwhere((ion_z_single >= -threshold_middle) & (ion_z_single <= threshold_middle))
            for middle_p in middle_points:
                try:
                    if np.sign(ion_z_single[middle_p-threshold_time]) != np.sign(ion_z_single[middle_p+threshold_time]):
                        permeation_event += 1
                        break
                except:
                    pass
            
    columb_e_conv = 6.242E18
    vmemb = 0.2
    conductance = permeation_event / columb_e_conv / (time * 1e-9) / vmemb * 1e12
    conductance_list.append(conductance)
ef_data['conductance_cla'] = conductance_list

In [ ]:
ef_data['note'] = ef_data['note'].apply(lambda x: 'EF_200' if x.find('EF_200')>=0 else 'EF_n200')

In [ ]:
ef_data.loc[ef_data.system == 2, 'ligand'] = 'NACHRA7_NOPNU_EPJ_POPC_nca'
ef_data.loc[ef_data.system == 3, 'ligand'] = 'NACHRA7_NOPNU_EPJ_POPC_nca'

In [ ]:
ef_data['conductance_sum'] = ef_data['conductance_sod'] + ef_data['conductance_cla']

In [ ]:
fig, ax = plt.subplots(figsize=(24,10))

l_box = sns.boxplot(x=ef_data['ligand'].to_numpy(),
            y=ef_data['conductance_sum'].to_numpy(),
            ax=ax,
            hue=ef_data['note'].to_numpy(),
            palette=['#6e6d59',cmap_open(cmap_open.N)],
            dodge=True,
            #width=0.6,
            linewidth=2.5,
            order=['NACHRA7_EPJ_POPC','NACHRA7_NOPNU_EPJ_POPC','NACHRA7_NOPNU_EPJ_POPC_nca', 'NACHRS7_NOPNU_EPJ_POPC_NOECD','NACHRS7_NOPNU_EPJ_POPC_NOICD','NACHRA7_E97A_NOPNU_EPJ_POPC'])
sns.swarmplot(x=ef_data['ligand'].to_numpy(),
              y=ef_data['conductance_sum'].to_numpy(),
              size=10,
              ax=ax,
              palette=[cmap_open(0),cmap_open(cmap_open.N)],
              hue=ef_data['note'].to_numpy(),
              dodge=True,
              order=['NACHRA7_EPJ_POPC','NACHRA7_NOPNU_EPJ_POPC','NACHRA7_NOPNU_EPJ_POPC_nca','NACHRS7_NOPNU_EPJ_POPC_NOECD','NACHRS7_NOPNU_EPJ_POPC_NOICD','NACHRA7_E97A_NOPNU_EPJ_POPC'])

ax.set_ylabel('Absolute Conductance (pS)')
ax.set_xticklabels(['Desensitizing \nw/o bound Ca$^{2+}$','Activating \nw/ bound Ca$^{2+}$', 'Activating \nw/o bound Ca$^{2+}$', 'Activating \nw/o ECD','Activating \nw/o ICD','Activating \nE97A'])
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[2:], ['200 mV', '-200 mV'])

l_box.artists[0].set_facecolor('#C78E52')
l_box.artists[1].set_facecolor('#C78E52')

set_axis_boarder(ax)
plt.savefig('Figures/figure_ef.pdf',bbox_inches = 'tight', pad_inches=0.1, transparent=False)